In [1]:
import pandas as pd
import seaborn as sns
sns.set(rc={'figure.figsize':(16,9)})



In [ ]:
df: pd.DataFrame = pd.read_csv("data/simple_autoencoder.csv")

In [ ]:
df["continual"] = pd.cut(df["n_experiences"], bins=[0, 1, 5], labels=[True, False])
sns.scatterplot(data=df, x="latent_dim", y="Accuracy", hue="continual")

In [10]:
from sklearn.decomposition import PCA
import dill
from torch.nn.functional import normalize
from torchvision.datasets import FashionMNIST
from torchvision import transforms
import torch

transform = transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = FashionMNIST(root="../datasets", train=True, download=True, transform=transform)
train_dataset = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=len(train_dataset))

images, labels = next(iter(train_dataset))


# dataset = "L128_FMNIST_SSAE.pkl"
# with open(dataset, "rb") as f:
#     dataset = dill.load(f)

In [9]:
# df = pd.DataFrame({"codes":images.tolist(), "class": labels})
images = images.flatten(dim=2)

tensor([-1., -1., -1.,  ..., -1., -1., -1.])

In [6]:
# df["codes"] = df["codes"].apply()
# pca = PCA(n_components=2)
# pca = pca.fit(list(df["codes"]))

df["codes"]

# for i, c in enumerate(list(pca.explained_variance_ratio_)):
#     print(f"Component {i} explains {c*100:.2f} of variance")


# df["dim_1"], df["dim_2"] = df["codes"].apply(lambda x: pca.transform([x])[0])

0        [[[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -...
1        [[[-1.0, -1.0, -1.0, -1.0, -1.0, -0.9921568632...
2        [[[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -...
3        [[[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -...
4        [[[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -...
                               ...                        
59995    [[[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -...
59996    [[[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -...
59997    [[[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -...
59998    [[[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -...
59999    [[[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -...
Name: codes, Length: 60000, dtype: object

In [ ]:
df["class"] = df["class"].apply(lambda x: str(x))
sns.scatterplot(data=df, x="dim_1", y="dim_2", hue="class", alpha=0.2).set_title('L128_FMNIST_SSAE')